In [52]:
import duckdb

In [53]:
conn = duckdb.connect('database/UserLogDB.db')

In [54]:
query = """DROP TABLE IF EXISTS users"""
conn.execute(query)
query = """DROP TABLE IF EXISTS logon_events"""
conn.execute(query)

In [55]:
query = """
CREATE TABLE IF NOT EXISTS users (
    userid VARCHAR(50) PRIMARY KEY,
    created_at TIMESTAMP
)
"""
conn.execute(query)

In [56]:
query = """
CREATE TABLE IF NOT EXISTS logon_events (
    userid VARCHAR(50),
    logon_ts TIMESTAMP
)
"""
conn.execute(query)

In [57]:
query = """SELECT * FROM logon_events"""
conn.execute(query).df()

,userid,logon_ts


In [58]:
query = """
INSERT INTO users (userid, created_at)
SELECT
    CONCAT('user', number['generate_series']),
    get_current_timestamp() - (CAST(RANDOM() * 2592000 AS INTEGER) || ' SECONDS')::INTERVAL
FROM GENERATE_SERIES(1, 10) AS number; 
"""
conn.execute(query)

In [59]:
query = """
INSERT INTO logon_events (userid, logon_ts)
SELECT userid, created_at + INTERVAL '1 DAY' AS logon_date
FROM users
WHERE userid in ('user2', 'user4', 'user3', 'user10');
"""
conn.execute(query)

In [60]:
query = """
INSERT INTO logon_events (userid, logon_ts)
SELECT userid, created_at + INTERVAL '2 DAY' AS logon_date
FROM users
WHERE userid in ('user1', 'user5', 'user4');

INSERT INTO logon_events (userid, logon_ts)
SELECT userid, created_at + INTERVAL '51 HOURS' AS logon_date
FROM users
WHERE userid in ('user5', 'user4');

INSERT INTO logon_events (userid, logon_ts)
SELECT userid, created_at + INTERVAL '3 DAY' AS logon_date
FROM users
WHERE userid in ('user1', 'user10', 'user9', 'user5', 'user6');
"""
conn.execute(query)

In [61]:
query = """SELECT * FROM users"""
conn.execute(query).df()

,userid,created_at
0,user1,2023-11-18 16:35:10.429
1,user2,2023-11-30 06:30:08.429
2,user3,2023-12-16 16:42:11.429
3,user4,2023-12-04 07:22:10.429
4,user5,2023-12-15 06:00:34.429
5,user6,2023-11-21 08:40:35.429
6,user7,2023-11-21 21:54:06.429
7,user8,2023-11-30 07:37:58.429
8,user9,2023-12-15 21:48:27.429
9,user10,2023-11-29 11:13:15.429


In [62]:
query = """SELECT * FROM logon_events"""
conn.execute(query).df()

,userid,logon_ts
0,user2,2023-12-01 06:30:08.429
1,user4,2023-12-05 07:22:10.429
2,user3,2023-12-17 16:42:11.429
3,user10,2023-11-30 11:13:15.429
4,user1,2023-11-20 16:35:10.429
5,user5,2023-12-17 06:00:34.429
6,user4,2023-12-06 07:22:10.429
7,user5,2023-12-17 09:00:34.429
8,user4,2023-12-06 10:22:10.429
9,user1,2023-11-21 16:35:10.429
